## Main Goal
The goal is to develop a dynamic pricing model that optimizes the prices of items to maximize revenue while remaining competitive in the market.  

**This involves:**

1. Analyzing the current pricing strategy and its impact on sales and revenue.
2. Comparing our pricing strategy with that of the competition to identify gaps and opportunities.
3. Developing a dynamic pricing model that adjusts prices based on factors such as competitor pricing, demand elasticity, and market trends.
4. Implementing and simulating the dynamic pricing model to compare its performance against the existing pricing strategy.

### Data Understanding
The given dataset contains pricing information for various items sold at a retail store and their corresponding prices at competing stores. The dataset includes the following columns:

1. **Fiscal_Week_Id**: The fiscal week identifier.
2. **Store_Id**: The store identifier.
3. **Item_Id**: The item identifier.
4. **Price**: The price of the item at our store.
5. **Item_Quantity**: The quantity of the item sold.
6. **Sales_Amount_No_Discount**: Sales amount without discount.
7. **Sales_Amount**: Sales amount after discounts.
8. **Competition_Price**: The price of the item at a competing store.

Key here is to unravel how our pricing strategies have influenced sales and revenue historically, and how competitor prices impact our performance. Considering factors like discounts and temporal aspects (thanks to the Fiscal_Week_Id) will help in identifying patterns and trends essential for optimizing prices. The ultimate goal is to find that sweet spot where price adjustments maximize revenue without sacrificing competitiveness.

In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv("datasets/Competition_Data.csv")

In [3]:
dataset.head()

,Index,Fiscal_Week_ID,Store_ID,Item_ID,Price,Item_Quantity,Sales_Amount_No_Discount,Sales_Amount,Competition_Price
0,0,2019-11,store_459,item_526,134.49,435,4716.74,11272.59,206.44
1,1,2019-11,store_459,item_526,134.49,435,4716.74,11272.59,158.01
2,2,2019-11,store_459,item_526,134.49,435,4716.74,11272.59,278.03
3,3,2019-11,store_459,item_526,134.49,435,4716.74,11272.59,222.66
4,4,2019-11,store_459,item_526,134.49,435,4716.74,11272.59,195.32


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Index                     100000 non-null  int64  
 1   Fiscal_Week_ID            100000 non-null  object 
 2   Store_ID                  100000 non-null  object 
 3   Item_ID                   100000 non-null  object 
 4   Price                     100000 non-null  float64
 5   Item_Quantity             100000 non-null  int64  
 6   Sales_Amount_No_Discount  100000 non-null  float64
 7   Sales_Amount              100000 non-null  float64
 8   Competition_Price         100000 non-null  float64
dtypes: float64(4), int64(2), object(3)
memory usage: 6.9+ MB


In [6]:
dataset.shape

(100000, 9)

In [7]:
dataset.isnull().sum()

Index                       0
Fiscal_Week_ID              0
Store_ID                    0
Item_ID                     0
Price                       0
Item_Quantity               0
Sales_Amount_No_Discount    0
Sales_Amount                0
Competition_Price           0
dtype: int64

In [8]:
dataset.isna().sum()

Index                       0
Fiscal_Week_ID              0
Store_ID                    0
Item_ID                     0
Price                       0
Item_Quantity               0
Sales_Amount_No_Discount    0
Sales_Amount                0
Competition_Price           0
dtype: int64